Goal: The primary goal of this notebook is to demonstrate and apply the QLoRA (Quantized Low-Rank Adaptation) technique to fine-tune a large pretrained language model (specifically, Bio_ClinicalBERT) for the task of multi-label adverse event classification from clinical text. The aim is to achieve efficient fine-tuning in terms of memory and computation while maintaining good model performance.

Method:

Data Preparation: A synthetic dataset of adverse event data with associated text and labels is loaded. The data is split into training, validation, and test sets. If a pre-defined 'split' column exists, it's used; otherwise, a stratified split is performed. The datasets are then converted to Hugging Face Dataset objects.


Model Loading and QLoRA Application: A pretrained Bio_ClinicalBERT model is loaded. 4-bit quantization is applied using BitsAndBytesConfig to reduce memory usage. LoRA layers are added to the model's query and value projections using LoraConfig and get_peft_model. The base model parameters are frozen, leaving only the LoRA parameters trainable.

Data Tokenization and Label Preparation: The text data in the datasets is tokenized using the model's tokenizer, with padding and truncation applied. The multi-labels (bleeding, fall, etc.) are prepared as a matrix of floating-point numbers, which is required for multi-label classification loss. Original text and label columns are removed after processing.



Training: A transformers.Trainer is set up with specific TrainingArguments (learning rate, batch size, epochs, weight decay, etc.). The Trainer is configured for multi-label classification and uses a custom compute_metrics function to calculate evaluation metrics like micro/macro F1, precision, recall, and Average Precision (AP) for each label and mean AP (mAP). The model is trained on the training set, with evaluation performed on the validation set at each epoch.
Evaluation and Saving: After training, the fine-tuned model is evaluated on the test set using the trainer.evaluate() method and the defined compute_metrics. The trained QLoRA adapter and tokenizer are saved to a specified output directory.


What is QLoRA?

QLoRA = Quantized Low-Rank Adaptation.

It makes fine-tuning large models more efficient by:

Quantization → reduces memory usage.

Low-rank adaptation → reduces computation needs.



Why use QLoRA?

Saves memory and computing power.

Keeps good model performance while making fine-tuning faster and cheaper.

What you’ll learn/do with QLoRA:

Understand its principles and advantages.

Apply it to large pretrained models.

Fine-tune efficiently for specific tasks.

Evaluate results with proper metrics.

Adjust settings (rank & quantization levels) to balance efficiency
vs. accuracy.





🛠 Step-by-Step Process

Prepare the dataset

Load and inspect the data.

Split into training, validation, and test sets.

Preprocess the text (cleaning, tokenization, padding).

Apply QLoRA to the model (next step after dataset prep).

Fine-tune the QLoRA-enhanced model.

Evaluate performance on unseen data.

Optimize settings for best efficiency and accuracy.

Spliting the data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Path to the synthetic data I provided earlier
CSV_PATH = "synthetic_ae_data.csv"

# Label columns for AE categories
LABEL_COLS = ["bleeding", "fall", "allergic_reaction", "hypoglycemia", "pressure_ulcer"]

data = pd.read_csv("/content/synthetic_ae_data_for__adverseeffect.csv")

if "split" in data.columns:
    # Use the provided split from the file
    train_data = data[data["split"] == "train"].reset_index(drop=True)
    val_data   = data[data["split"] == "valid"].reset_index(drop=True)
    test_data  = data[data["split"] == "test"].reset_index(drop=True)
else:
    # Recreate a 70/15/15 split with stratification on any_positive label
    data["any_positive"] = (data[LABEL_COLS].sum(axis=1) > 0).astype(int)
    train_data, temp_data = train_test_split(
        data, test_size=0.30, random_state=42, stratify=data["any_positive"]
    )
    val_data, test_data = train_test_split(
        temp_data, test_size=0.50, random_state=42, stratify=temp_data["any_positive"]
    )
    # Reset indices after splitting
    train_data = train_data.reset_index(drop=True)
    val_data = val_data.reset_index(drop=True)
    test_data = test_data.reset_index(drop=True)
    # Optional: drop helper column
    for df in (train_data, val_data, test_data):
        if "any_positive" in df.columns:
            df.drop(columns=["any_positive"], inplace=True)


print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")

# Optional: quick class balance check
def balance(df):
    return {c: float(df[c].mean()) for c in LABEL_COLS}

print("Train pos rates:", balance(train_data))
print("Valid pos rates:", balance(val_data))
print("Test pos rates:", balance(test_data))

Training set size: 12
Validation set size: 4
Test set size: 4
Train pos rates: {'bleeding': 0.25, 'fall': 0.25, 'allergic_reaction': 0.25, 'hypoglycemia': 0.25, 'pressure_ulcer': 0.16666666666666666}
Valid pos rates: {'bleeding': 0.25, 'fall': 0.25, 'allergic_reaction': 0.0, 'hypoglycemia': 0.0, 'pressure_ulcer': 0.25}
Test pos rates: {'bleeding': 0.0, 'fall': 0.25, 'allergic_reaction': 0.25, 'hypoglycemia': 0.25, 'pressure_ulcer': 0.25}


Explanation

We split the dataset into three subsets: training (70 percent), validation (15 percent), and test (15 percent) to ensure we can monitor the model’s performance during and after training. The training data will be used to fine-tune the model, while the validation data helps avoid overfitting, and the test data evaluates the final model.



🔑 Key Points on Model Training & Validation

Monitoring Performance

Track how well the model does on validation data during training.

If training accuracy improves but validation accuracy stagnates or worsens → the model is overfitting.

Early Stopping

Stop training when validation performance no longer improves.

Prevents the model from memorizing training data and losing generalization ability.

Hyperparameter Tuning

Use the validation set to adjust settings like learning rate or regularization strength.

Choose the settings that give the best validation performance.

Model Selection

When testing multiple models or fine-tuning strategies, compare them using validation performance.

Select the best-performing model for the task.

The split column is a label that tells you which subset each row belongs to: train, valid (validation), or test.

Why it’s used

    Reproducibility: Everyone uses the same fixed split instead of making a new random split each run.
    No data leakage: Ensures examples used to tune the model (train/valid) never appear in the final evaluation (test).
    Proper workflow:
        Train set: fit the model.
        Validation set: choose hyperparameters, thresholds, do early stopping.
        Test set: one-time, unbiased performance estimate after everything is fixed.

How to use it

    Filter by the split value instead of calling train_test_split:
        train = df[df["split"] == "train"]
        val = df[df["split"] == "valid"]
        test = df[df["split"] == "test"]


🔑 Key Points – Applying QLoRA

Load a pretrained model

Start with an existing large model (e.g., BERT or GPT).

Apply quantization

Compress the model to reduce memory usage while keeping performance strong.

Add LoRA (Low-Rank Adaptation)

Insert LoRA into specific layers like attention heads or feed-forward networks.

These layers will be fine-tuned, not the whole model.

Freeze the base model

Set most parameters to not update during training (requires_grad = False).

Only LoRA parameters stay trainable (requires_grad = True).

Why this matters

Saves memory and compute.

Fine-tunes only the small added LoRA matrices, not the whole huge model.

1) Install dependencies1)

In [ ]:
!pip -q install "transformers>=4.41.0" "peft>=0.11.1" "accelerate>=0.28.0" "bitsandbytes>=0.43.1" \
                 "scikit-learn>=1.2.0" pandas numpy

In [ ]:
pip -q install -U "transformers>=4.41.2" "accelerate>=0.31.0" "peft>=0.11.1" "bitsandbytes>=0.43.1" "triton>=2.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.0 MB/s eta 0:00:00


In [ ]:
import torch, importlib
print("Torch:", torch.__version__, "CUDA available:", torch.cuda.is_available())
print("bf16 supported:", torch.cuda.is_bf16_supported())
print("Have triton?", importlib.util.find_spec("triton") is not None)
print("Have bitsandbytes?", importlib.util.find_spec("bitsandbytes") is not None)

Torch: 2.6.0+cu124 CUDA available: True
bf16 supported: True
Have triton? True
Have bitsandbytes? True


2) Check GPU and environment

In [ ]:
import  sys, platform
!nvidia-smi || echo "No NVIDIA GPU visible"
print("Torch version:", torch.__version__, "CUDA available:", torch.cuda.is_available())
print("Python:", sys.version.split()[0], "OS:", platform.platform())
if not torch.cuda.is_available():
    raise SystemExit("Please enable a GPU runtime in Colab (Runtime -> Change runtime type -> GPU).")

Tue Aug 19 04:15:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P0             28W /   70W |     104MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Pick a model

    Short notes (<=512 tokens): Bio_ClinicalBERT
    Long notes: Clinical-Longformer (4096 tokens, heavier)


In [ ]:
MODEL_NAME = "emilyalsentzer/Bio_ClinicalBERT"  # or "yikuan8/Clinical-Longformer"
MAX_LENGTH = 512  # set 4096 for Longformer if you switch models
LABEL_COLS = ["bleeding","fall","allergic_reaction","hypoglycemia","pressure_ulcer"]

Load 4-bit quantized model and add LoRA (QLoRA)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig

from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

# Pick a model (change to "yikuan8/Clinical-Longformer" for long notes and set max_length=4096)
MODEL_NAME = "emilyalsentzer/Bio_ClinicalBERT"
NUM_LABELS = 5  # bleeding, fall, allergic_reaction, hypoglycemia, pressure_ulcer

# Use bf16 if supported (you have it)
compute_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

# 4-bit quantization config (avoids deprecated load_in_4bit/load_in_8bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=compute_dtype,
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification",
    quantization_config=bnb_config,   # <- correct way (no deprecation warning)
    torch_dtype=compute_dtype,
    device_map="auto",
)

# Prepare for k-bit training and add LoRA
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

# For BERT-like encoders, target query/value projections
peft_cfg = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query", "value"],
)
model = get_peft_model(model, peft_cfg)
model.print_trainable_parameters()

print("bf16?", compute_dtype == torch.bfloat16)
print("Device map:", getattr(model, "hf_device_map", None))
print("QLoRA setup complete.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 593,669 || all params: 108,907,786 || trainable%: 0.5451
bf16? True
Device map: {'': 0}
QLoRA setup complete.


8) Tokenize the text and level

In [ ]:
import numpy as np
from datasets import Dataset

def tokenize_and_prepare_labels(text_batch, bleeding_batch, fall_batch, allergic_reaction_batch, hypoglycemia_batch, pressure_ulcer_batch):
    # Reconstruct the examples dictionary from the positional arguments
    examples = {
        "text": text_batch,
        "bleeding": bleeding_batch,
        "fall": fall_batch,
        "allergic_reaction": allergic_reaction_batch,
        "hypoglycemia": hypoglycemia_batch,
        "pressure_ulcer": pressure_ulcer_batch,
    }

    # Tokenize the text
    tokenized_inputs = tokenizer(
        examples["text"],
        max_length=MAX_LENGTH,
        padding="max_length",
        truncation=True,
    )

    # Prepare the labels as a list of lists of floats
    labels_batch = {k: examples[k] for k in LABEL_COLS}
    labels_matrix = np.array([labels_batch[label] for label in LABEL_COLS], dtype=np.float32).T # Convert to float32
    tokenized_inputs["labels"] = labels_matrix.tolist()

    return tokenized_inputs

# Recreate Hugging Face Datasets from DataFrames to ensure original columns are present
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

# Apply the tokenization and label preparation to the datasets
# Explicitly specify input_columns to ensure 'text' and label columns are available
train_dataset = train_dataset.map(tokenize_and_prepare_labels, batched=True, input_columns=["text"] + LABEL_COLS)
val_dataset = val_dataset.map(tokenize_and_prepare_labels, batched=True, input_columns=["text"] + LABEL_COLS)
test_dataset = test_dataset.map(tokenize_and_prepare_labels, batched=True, input_columns=["text"] + LABEL_COLS)

# Remove original columns that are no longer needed by the model
train_dataset = train_dataset.remove_columns(["text", "doc_id", "split"] + LABEL_COLS)
val_dataset = val_dataset.remove_columns(["text", "doc_id", "split"] + LABEL_COLS)
test_dataset = test_dataset.remove_columns(["text", "doc_id", "split"] + LABEL_COLS)


print(train_dataset)
print(val_dataset)
print(test_dataset)

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset({
    features: ['labels_multi', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 12
})
Dataset({
    features: ['labels_multi', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 4
})
Dataset({
    features: ['labels_multi', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 4
})


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, average_precision_score
from transformers import Trainer, TrainingArguments
# import torch # No longer need to import torch just for .cpu()

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Move tensors to CPU before converting to NumPy - Removed as inputs are already numpy arrays
    # logits_cpu = logits.cpu()
    # labels_cpu = labels.cpu()

    # Calculate probabilities using the sigmoid function
    probs = 1 / (1 + np.exp(-logits))
    # Convert probabilities to predictions based on threshold 0.5
    preds = (probs >= 0.5).astype(int)

    # Calculate metrics
    micro_p, micro_r, micro_f1, _ = precision_recall_fscore_support(labels, preds, average="micro", zero_division=0)
    macro_p, macro_r, macro_f1, _ = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
    # Calculate Average Precision for each label and then the mean Average Precision (mAP)
    ap = [average_precision_score(labels[:,i], probs[:,i]) for i in range(labels.shape[1])]

    return {
        "micro_f1": micro_f1, "macro_f1": macro_f1,
        "micro_precision": micro_p, "micro_recall": micro_r,
        "macro_precision": macro_p, "macro_recall": macro_r,
        "map_macro_pr_auc": float(np.mean(ap)),
        **{f"ap_{LABEL_COLS[i]}": float(ap[i]) for i in range(len(LABEL_COLS))}
    }


args = TrainingArguments(
    output_dir="outputs/qlora-ae",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,            # keep small for Colab
    weight_decay=0.01,
    warmup_ratio=0.06,
    logging_steps=20,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    bf16=(compute_dtype==torch.bfloat16),
    fp16=(compute_dtype==torch.float16),
    seed=42,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset, # Use Hugging Face Dataset
    eval_dataset=val_dataset,   # Use Hugging Face Dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-2451329440.py:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Micro F1,Macro F1,Micro Precision,Micro Recall,Macro Precision,Macro Recall,Map Macro Pr Auc,Ap Bleeding,Ap Fall,Ap Allergic Reaction,Ap Hypoglycemia,Ap Pressure Ulcer
1,No log,0.627431,0.181818,0.080000,0.125000,0.333333,0.050000,0.200000,0.400000,0.500000,0.500000,0.000000,0.000000,1.000000
2,No log,0.609518,0.222222,0.133333,0.166667,0.333333,0.100000,0.200000,0.500000,0.500000,1.000000,0.000000,0.000000,1.000000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  wa

TrainOutput(global_step=4, training_loss=0.6148951649665833, metrics={'train_runtime': 4.829, 'train_samples_per_second': 4.97, 'train_steps_per_second': 0.828, 'total_flos': 6358605447168.0, 'train_loss': 0.6148951649665833, 'epoch': 2.0})

Evaluate on test set and save adapter

In [ ]:
metrics = trainer.evaluate(test_dataset)
print("Test metrics:", metrics)

# Save LoRA adapter and tokenizer
adapter_dir = "outputs/qlora-ae"
model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(adapter_dir)
print("Saved adapter to", adapter_dir)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


Test metrics: {'eval_loss': 0.6510210037231445, 'eval_micro_f1': 0.46153846153846156, 'eval_macro_f1': 0.36, 'eval_micro_precision': 0.3333333333333333, 'eval_micro_recall': 0.75, 'eval_macro_precision': 0.3, 'eval_macro_recall': 0.6, 'eval_map_macro_pr_auc': 0.45, 'eval_ap_bleeding': 0.0, 'eval_ap_fall': 0.5, 'eval_ap_allergic_reaction': 1.0, 'eval_ap_hypoglycemia': 0.25, 'eval_ap_pressure_ulcer': 0.5, 'eval_runtime': 0.3163, 'eval_samples_per_second': 12.645, 'eval_steps_per_second': 3.161, 'epoch': 2.0}
Saved adapter to outputs/qlora-ae


Inference demo on new text

In [ ]:
import torch
import numpy as np

id2label = {i: c for i,c in enumerate(LABEL_COLS)}
def predict(text, threshold=0.5):
    enc = tokenizer(text, return_tensors="pt", truncation=True, max_length=MAX_LENGTH, padding="max_length").to(model.device)
    with torch.no_grad():
        logits = model(**enc).logits.detach().cpu().numpy()[0]
    probs = 1/(1+np.exp(-logits))
    preds = (probs >= threshold)
    return {id2label[i]: float(probs[i]) for i in range(len(LABEL_COLS))}, [id2label[i] for i in range(len(LABEL_COLS)) if preds[i]]

sample = "Patient slipped and fell in the bathroom. Later developed melena with ongoing bleeding. Treated for hypoglycemia overnight."
probs, labels = predict(sample, threshold=0.5)
print("Probabilities:", probs)
print("Predicted labels:", labels)

Probabilities: {'bleeding': 0.3293917179107666, 'fall': 0.5203744769096375, 'allergic_reaction': 0.3225259780883789, 'hypoglycemia': 0.5252470970153809, 'pressure_ulcer': 0.29828691482543945}
Predicted labels: ['fall', 'hypoglycemia']


Inference Demo: A simple function is provided to demonstrate how to load the fine-tuned model and tokenizer and make predictions on new text samples, outputting probabilities and predicted labels based on a threshold.
Key Result/Finding (Model Performance): The notebook successfully demonstrates the process of applying QLoRA for fine-tuning. The training and evaluation steps run, and metrics are computed and printed. The test set evaluation metrics are provided, including:

eval_loss: 0.651
eval_micro_f1: 0.462
eval_macro_f1: 0.360
eval_map_macro_pr_auc: 0.450
Individual Average Precision scores for each label.
These metrics indicate that the fine-tuned model has some ability to classify adverse events, but the performance (especially the macro F1 and mAP) is relatively low, which is expected given the small synthetic dataset size and limited training epochs. The individual AP scores show variability across different adverse event categories.

Suggestions for Improvement: The notebook itself already includes several excellent suggestions:

Handle Longer Notes: Switch to the yikuan8/Clinical-Longformer model and increase MAX_LENGTH to 4096 for better handling of longer clinical text, if VRAM allows.
Increase Dataset Size and Epochs: For better performance, use a larger, more representative dataset and train for more epochs if the Colab runtime limits permit.
Calibrate Per-Class Thresholds: Optimize the classification threshold (currently fixed at 0.5) for each individual adverse event class on the validation set to potentially maximize per-class F1 scores.
Address Bitsandbytes Installation Issues: The notebook encountered persistent issues with bitsandbytes installation. Resolving these (e.g., by trying different versions, ensuring environment compatibility, or using a Colab runtime where it's pre-installed or easily installable) is crucial for using 4-bit quantization reliably. (While 8-bit was attempted, the root cause of the bitsandbytes issue prevented it from working as well).



    To handle very long notes end-to-end, switch to yikuan8/Clinical-Longformer and set MAX_LENGTH=4096 (requires more VRAM).
    For better results, increase dataset size and training epochs if Colab time allows.
    You can calibrate per-class thresholds on the validation set by sweeping from 0.1 to 0.9 to maximize F1 per class.


Overall, the notebook provides a clear and functional example of applying QLoRA for multi-label text classification, highlighting the efficiency benefits and the typical workflow, while also pointing out areas for practical improvement to achieve higher performance on a real-world task.